# Animal Recognition with Neural Networks

## Libraries import

In [1]:
animals_wanted = ["Bear", "Chicken", "Turtle"]

In [2]:
#                                                   LIBRARIES

import os

# Dataset
from kaggle.api.kaggle_api_extended import KaggleApi
from datasets import load_dataset

# Image preprocesssing- processing
import cv2
from PIL import Image
import numpy as np

# Neural Network libraries
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout

In [3]:
train_animals_reduced=[]
test_animals_reduced=[]
# supervised -> labels are needed
train_labels_reduced=[]
test_labels_reduced=[]

## Prepare Dataset

Transform images to the right format for the model

2 arrays:
- data: array of images converted to numpy array
- labels: corresponding labels
    1. Dog
    2. Cat
    ...
    - Zebra
    - Lion
    - Leopard
    - Cheetah
    - Tiger
    - Bear
    - Brown Bear
    - Butterfly
    - Canary
    - Crocodile
    - Polar Bear
    - Bull
    - Camel
    - Crab
    - Chicken
    - Centipede
    - Cattle
    - Caterpillar
    - Duck

In [4]:
train_animals_reduced=[]
test_animals_reduced=[]
# supervised -> labels are needed
train_labels_reduced=[]
test_labels_reduced=[]

In [24]:
#                                           Transform images into an array

def load_and_save_data_reduced(set_tt):
    assert set_tt == "test" or "train"
    folder_directory = ""
    if set_tt == "train":
        folder_directory = "../datasets/animals/animals-detection-images-dataset" + os.sep + "train"
    elif set_tt == "test":
        folder_directory = "../datasets/animals/animals-detection-images-dataset" + os.sep + "test"
    print("============= " + set_tt + " =============")
    l_data=[]
    # supervised -> labels are needed
    l_labels=[]

    label = 0

    animals_list=os.listdir(folder_directory)

    # for each animal directory in the dataset directory
    for animal in  animals_list:

        if animal in animals_wanted:
            animal_imgs = os.listdir(folder_directory + os.sep + animal)

            num_images = 0

            # for each image of the current image
            # print(animal_dir)
            for image in animal_imgs:
                # read image
                img_dir = folder_directory + os.sep + animal + os.sep + image
                if os.path.isfile(img_dir):
                    img=cv2.imread(img_dir)

                    # convert image to array
                    img = np.array(img)
                    img_from_ar = Image.fromarray(img, 'RGB')

                    # image of same size needed for CNN, ensure they are -> 50X50
                    resized_image = img_from_ar.resize((50, 50))

                    # convert back to numpy array
                    l_data.append(np.array(resized_image))

                    l_labels.append(label)
                    num_images += 1


            print("Label " + str(label) + " = " + str(animal) + " with " + str(num_images) + " images")


            # increment label for next animal
            label += 1

    global train_animals_reduced, train_labels_reduced, test_animals_reduced, test_labels_reduced
    # Transform "normal" array into numpy array
    if set_tt == "test":
        test_animals_reduced=np.array(l_data)
        test_labels_reduced=np.array(l_labels)

        # save numpy arrays so manipulation dont need to be done again
        np.save("test_animals_reduced",test_animals_reduced)
        np.save("test_labels_reduced",test_labels_reduced)
    elif set_tt == "train":
        train_animals_reduced=np.array(l_data)
        train_labels_reduced=np.array(l_labels)

        # save numpy arrays so manipulation dont need to be done again
        np.save("train_animals_reduced",train_animals_reduced)
        np.save("train_labels_reduced",train_labels_reduced)

def load_data_and_labels_reduced():
    global train_animals_reduced, train_labels_reduced, test_animals_reduced, test_labels_reduced
    train_animals_reduced = np.load("train_animals_reduced.npy")
    train_labels_reduced = np.load("train_labels_reduced.npy")
    test_animals_reduced = np.load("test_animals_reduced.npy")
    test_labels_reduced = np.load("test_labels_reduced.npy")

In [14]:
# if its not saved yet:
#load_and_save_data_reduced()

# if its saved:
#load_data_and_labels_reduced()



In [25]:
load_and_save_data_reduced("train")
load_and_save_data_reduced("test")

============= train =============
Label 0 = Bear with 87 images
Label 1 = Chicken with 388 images
Label 2 = Turtle with 24 images
============= test =============
Label 0 = Bear with 39 images
Label 1 = Chicken with 137 images
Label 2 = Turtle with 5 images


#### ============= train =============
Label 0 = Bear with 87 images
Label 1 = Chicken with 388 images
Label 2 = Duck with 542 images
Label 3 = Frog with 588 images
Label 4 = Sea turtle with 239 images
Label 5 = Squirrel with 367 images

#### ============= test =============
Label 0 = Bear with 39 images
Label 1 = Chicken with 137 images
Label 2 = Duck with 88 images
Label 3 = Frog with 77 images
Label 4 = Sea turtle with 87 images
Label 5 = Squirrel with 68 images

In [8]:
load_and_save_data_reduced("train")
load_and_save_data_reduced("test")
#load_data_and_labels_reduced()

============= train =============
Label 0 = Bear with 87 images
Label 1 = Chicken with 388 images
Label 2 = Duck with 542 images
Label 3 = Frog with 588 images
Label 4 = Sea turtle with 239 images
Label 5 = Squirrel with 367 images
============= test =============
Label 0 = Bear with 39 images
Label 1 = Chicken with 137 images
Label 2 = Duck with 88 images
Label 3 = Frog with 77 images
Label 4 = Sea turtle with 87 images
Label 5 = Squirrel with 68 images


In [16]:
# shuffle animals and labels from both sets
s=np.arange(train_animals_reduced.shape[0])
np.random.shuffle(s)
train_animals_reduced=train_animals_reduced[s]
train_labels_reduced=train_labels_reduced[s]

s=np.arange(test_animals_reduced.shape[0])
np.random.shuffle(s)
test_animals_reduced=test_animals_reduced[s]
test_labels_reduced=test_labels_reduced[s]

In [17]:
# number of classes (labels) and size of dataset
num_classes_train=len(np.unique(train_labels_reduced))
num_classes_test=len(np.unique(test_labels_reduced))
assert num_classes_test == num_classes_train
num_classes = num_classes_train

train_data_length=len(train_animals_reduced)
test_data_length=len(test_animals_reduced)
data_length = train_data_length + test_data_length
print("Train data length: " + str(train_data_length))
print("Test data length: " + str(test_data_length))

Train data length: 499
Test data length: 181


In [18]:
# /255
train_animals_reduced = train_animals_reduced.astype('float32')/255
test_animals_reduced = test_animals_reduced.astype('float32')/255

In [12]:
# One hot encoding for labels
train_labels_reduced=keras.utils.to_categorical(train_labels_reduced,num_classes_train)
test_labels_reduced=keras.utils.to_categorical(test_labels_reduced,num_classes_test)

In [13]:
# save numpy arrays so manipulation dont need to be done again
np.save("test_animals",test_animals_reduced)
np.save("test_labels",test_labels_reduced)

np.save("train_animals",train_animals_reduced)
np.save("train_labels",train_labels_reduced)